In [7]:
import sqlite3
import pandas as pd
import pickle
from src.DataPreprocessing.DataPreprocessing import DataPreprocessing
from src.FeatureEngineering.FeatureEngineering import FeatureEngineering
from src.PipelineSettings.PipelineSettings import PipelineSettings

In [4]:
# Read in all sheets from the .xlsx-file -> See ChatGPT to write this using a for loop and dictionary
oil_futures = pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='oil_futures').set_index('Date')
gasoline_futures= pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='gasoline_futures').set_index('Date')
heating_oil_futures= pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='heating_oil_futures').set_index('Date')
oil_futures_new= pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='oil_futures_new').set_index('Date')
heating_gasoline_futures= pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='heating_gasoline_futures').set_index('Date')
crush_spread_futures= pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='crush_spread_futures').set_index('Date')
spark_spread_futures= pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='spark_spread_futures').set_index('Date')
diverse_futures= pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='diverse_futures').set_index('Date')
governement_bonds_1= pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='governement_bonds_1').set_index('Date')
governement_bonds_2= pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='governement_bonds_2').set_index('Date')
governement_bonds_3= pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='governement_bonds_3').set_index('Date')
metals = pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='metals').set_index('Date')
bond_indices = pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='bond_indices').set_index('Date')
emmission = pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='emmission').set_index('Date')
lumber = pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='Lumber').set_index('Date')
coffee = pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='coffee').set_index('Date')
corn = pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='corn').set_index('Date')
test = pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='test').set_index('Date')
gold = pd.read_excel('src/data/data/data_PJA.xlsx', decimal=',', parse_dates=True, sheet_name='gold').set_index('Date')




# Merge into one big dataframe
dataframes = \
    [
    oil_futures, gasoline_futures, heating_oil_futures, oil_futures_new, heating_gasoline_futures,
    crush_spread_futures, spark_spread_futures, diverse_futures, governement_bonds_1, metals,
    bond_indices, emmission, lumber, coffee, corn, test, governement_bonds_2, governement_bonds_3,
    gold
    ]
data = pd.DataFrame(columns=['Date']).set_index('Date')
for frame in dataframes:
    data = data.merge(frame, left_index=True, right_index=True, how='outer')
    duplicates = data.filter(like='_y', axis=1)
    data = data.drop(columns=duplicates.columns)
    data.columns = data.columns.str.replace('_x', '')

# Perform train-test-split
data_train = data[data.index < '2019-01-01']
data_test = data[data.index >= '2019-01-01']

In [10]:
# Load Pairs mapping and code-name mapping
pairs_mapping = pd.read_excel('src/data/data/codes_mapping.xlsx', sheet_name='mapping')
all_pairs_train = pd.DataFrame()
for pair in range(0,len(pairs_mapping)):
    ps = PipelineSettings().load_settings()
    data_ = DataPreprocessing().load_sql(pairs_mapping.loc[pair,'asset_1'], pairs_mapping.loc[pair,'asset_2'])
    fs = FeatureEngineering(data_, ps)
    features = fs.run_feature_engineering(params_method='Kalman', smoothing=True)
    features['pair'] = pair
    all_pairs_train = pd.concat([all_pairs_train, features])
all_pairs_normalized_train = fs.scale_features(all_pairs_train)[0]
all_pairs_test = pd.DataFrame()
for pair in range(0,len(pairs_mapping)):
    ps = PipelineSettings().load_settings()
    data_ = DataPreprocessing().load_sql(pairs_mapping.loc[pair,'asset_1'], pairs_mapping.loc[pair,'asset_2'], table='test_set')
    fs = FeatureEngineering(data_, ps)
    features = fs.run_feature_engineering(params_method='Kalman', smoothing=True)
    features['pair'] = pair
    all_pairs_test = pd.concat([all_pairs_test, features])
with open('src/Output/scalers/_scaler_2023-06-29_23:10:26.pkl', 'rb') as f:
    scaler = pickle.load(f)   
all_pairs_normalized_test = fs.scale_features(all_pairs_train, scaler=scaler)[0]



/Users/PJA/anaconda3/lib/python3.10/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/PJA/anaconda3/lib/python3.10/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/PJA/anaconda3/lib/python3.10/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/PJA/anaconda3/lib/python3.10/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/PJA/anaconda3/lib/python3.10/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/PJA/anaconda3/lib/python3.10/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: invalid value encounte

Applying Scaler passed as argument


In [ ]:
all_pairs = pd.DataFrame()
for pair in range(0,len(pairs_mapping)):
    ps = PipelineSettings().load_settings()
    data_ = DataPreprocessing().load_sql(pairs_mapping.loc[pair,'asset_1'], pairs_mapping.loc[pair,'asset_2'])
    fs = FeatureEngineering(data_, ps)
    features = fs.run_feature_engineering(params_method='Kalman', smoothing=True)
    features['pair'] = pair
    all_pairs = pd.concat([all_pairs, features])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [ ]:
all_pairs_normalized_train = fs.scale_features(all_pairs_train)[0]

In [ ]:
all_pairs_test = pd.DataFrame()
for pair in range(0,len(pairs_mapping)):
    ps = PipelineSettings().load_settings()
    data_ = DataPreprocessing().load_sql(pairs_mapping.loc[pair,'asset_1'], pairs_mapping.loc[pair,'asset_2'], table='test_set')
    fs = FeatureEngineering(data_, ps)
    features = fs.run_feature_engineering(params_method='Kalman', smoothing=True)
    features['pair'] = pair
    all_pairs_test = pd.concat([all_pairs_test, features])

NameError: name 'pd' is not defined

In [4]:
import pickle
output_dir = "../Output/"
time = 'l'
with open(f'{output_dir}nets/trained_net_{time}.pkl', 'wb') as f:
            pickle.dump(1, f)
with open(f'{output_dir}stats/training_stats_{time}.pkl', 'wb') as f:
            pickle.dump(2, f)

In [11]:
# Connect to database
conn = sqlite3.connect('src/data/data/pairs_trading.db')
cursor = conn.cursor()

# Store time series in database
# data.to_sql('time_series', conn, if_exists='fail', index=True)
# data_test.to_sql('test_set', conn, if_exists='fail', index=True)
# data_train.to_sql('train_set', conn, if_exists='fail', index=True)
# pairs_mapping.to_sql('pairs_mapping', conn, if_exists='fail', index=False)
all_pairs_normalized_train.to_sql('training_paires', conn, if_exists='replace', index=False)
all_pairs_normalized_test.to_sql('training_paires', conn, if_exists='replace', index=False)

# Store pairs mapping for training in database table


# Commit the changes to the database and close the connection
conn.commit()
conn.close()